In [33]:
import pandas as pd
df = pd.read_excel('data/Educação.xlsx')
resultado = df 
resultado.columns


Index(['Município', 'IDHM 2010 ', 'População (Censo 2022)', 'Território',
       'IDEB AI - Pública', 'IDEB Anos Finais - Pública',
       'SAEPI Português 2º ano Ensino Fundamental ',
       'SAEPI Matemática 2º ano Ensino Fundamental ',
       'Taxa de evasão ensino fund. Anos Iniciais',
       'Taxa de evasão ensino fund. Anos Finais ',
       'Taxa de Abandono - Anos Iniciais', 'Taxa de Abandono - Anos Finais'],
      dtype='object')

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# Assumindo que sua variável se chama 'resultado'
df = resultado.copy()

# Remover espaços extras nos nomes das colunas
df.columns = [col.rstrip() for col in df.columns]

# Função para tratar valores como "#VALUE!" e converter strings com vírgula para float
def tratar_valor(x):
    if isinstance(x, str):
        if x == "#VALUE!":
            return np.nan  # Usar NaN para depois preencher com a média
        # Substituir vírgula por ponto para números em formato brasileiro
        return float(x.replace(',', '.'))
    return x

# Definir os indicadores para análise (excluindo IDHM 2010, SAEPI, Território e População)
indicadores_para_analise = [
    'IDEB AI - Pública',
    'IDEB Anos Finais - Pública',
    'Taxa de evasão ensino fund. Anos Iniciais',
    'Taxa de evasão ensino fund. Anos Finais',
    'Taxa de Abandono - Anos Iniciais',
    'Taxa de Abandono - Anos Finais'
]

# Converter valores para numérico
for col in indicadores_para_analise:
    if col in df.columns:
        df[col] = df[col].apply(tratar_valor)
    else:
        # Buscar coluna com espaço extra no final
        matching_col = [c for c in df.columns if c.rstrip() == col]
        if matching_col:
            df[col] = df[matching_col[0]].apply(tratar_valor)
            df = df.drop(columns=matching_col[0])

# Tratar valores NaN nas colunas de indicadores
for col in indicadores_para_analise:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())

# Preparar dados para clusterização
# Usamos apenas as colunas selecionadas
df_for_clustering = df[indicadores_para_analise].copy()

# Verificar se temos dados válidos para todas as colunas
print("Estatísticas dos dados antes da normalização:")
print(df_for_clustering.describe())

# 1. Verificar a distribuição dos dados
plt.figure(figsize=(15, 8))
for i, col in enumerate(indicadores_para_analise):
    plt.subplot(2, 3, i+1)
    sns.histplot(df_for_clustering[col], kde=True)
    plt.title(col)
    plt.tight_layout()
plt.savefig('distribuicao_original.png')
plt.close()

# 2. Transformação logarítmica para lidar com assimetria
# Para valores negativos ou zero, usamos log1p (adiciona 1 antes do log)
df_log = df_for_clustering.copy()
for col in df_log.columns:
    # Verificar se temos valores negativos
    min_value = df_log[col].min()
    if min_value < 0:
        # Para colunas com valores negativos, deslocar para tornar todos positivos
        shift = abs(min_value) + 1e-10
        df_log[col] = np.log1p(df_log[col] + shift)
    else:
        # Para colunas sem valores negativos
        df_log[col] = np.log1p(df_log[col])

# 3. Normalização após transformação log
scaler = StandardScaler()
df_normalized = pd.DataFrame(
    scaler.fit_transform(df_log),
    columns=df_log.columns
)

# Verificar boxplot após normalização
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_normalized)
plt.title('Boxplot após Log+StandardScaler - Verificar outliers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('outliers_pos_log.png')
plt.close()

# 4. Lidar com outliers remanescentes
for col in df_normalized.columns:
    # Limitar valores a ±3 desvios padrão
    df_normalized[col] = df_normalized[col].clip(lower=-3, upper=3)

# 5. Determinar número ideal de clusters
wcss = []
silhouette_scores = []
range_clusters = range(2, 8)  # Testando de 2 a 7 clusters

for i in range_clusters:
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(df_normalized)
    wcss.append(kmeans.inertia_)
    
    # Calcular silhouette score
    silhouette_scores.append(silhouette_score(df_normalized, kmeans.labels_))

# Plotar o "Elbow Method" e Silhouette Score
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range_clusters, wcss, marker='o')
plt.title('Método do Cotovelo (Elbow Method)')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.axvline(x=5, color='r', linestyle='--', label='Selecionado: 5 clusters')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range_clusters, silhouette_scores, marker='o')
plt.title('Método da Silhueta')
plt.xlabel('Número de Clusters')
plt.ylabel('Silhouette Score')
plt.axvline(x=5, color='r', linestyle='--', label='Selecionado: 5 clusters')
plt.legend()
plt.tight_layout()
plt.savefig('n_clusters.png')
plt.close()

# 6. Aplicar K-Means com 5 clusters
n_clusters = 5  # Definido diretamente como 5

kmeans = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=300, n_init=10, random_state=42)
df['cluster'] = kmeans.fit_predict(df_normalized)

# 7. Visualização com PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(df_normalized)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['cluster'] = df['cluster']

plt.figure(figsize=(10, 8))
sns.scatterplot(x='PC1', y='PC2', hue='cluster', data=pca_df, palette='viridis')
plt.title('Visualização dos 5 Clusters com PCA')
plt.tight_layout()
plt.savefig('clusters_pca.png')
plt.close()

# 8. Análise das características dos clusters
cluster_stats = df.groupby('cluster')[indicadores_para_analise].agg(['mean', 'median', 'min', 'max', 'count'])
print("Estatísticas dos clusters (valores originais):")
print(cluster_stats)

# Contagem de municípios por cluster
print("\nQuantidade de municípios em cada cluster:")
contagem_por_cluster = df.groupby('cluster')['Município'].count()
for cluster, count in contagem_por_cluster.items():
    print(f"Cluster {cluster}: {count} municípios")

# 9. Dados completos de cada município por cluster
print("\nDados completos de cada município por cluster:")
for cluster_num in sorted(df['cluster'].unique()):
    print(f"\n=== CLUSTER {cluster_num} ===")
    cluster_data = df[df['cluster'] == cluster_num][['Município'] + indicadores_para_analise]
    print(cluster_data.to_string(index=False))
    
# Salvar dados detalhados por cluster em arquivos separados
for cluster_num in sorted(df['cluster'].unique()):
    cluster_data = df[df['cluster'] == cluster_num][['Município'] + indicadores_para_analise]
    cluster_data.to_csv(f'cluster_{cluster_num}_detalhes.csv', index=False)

# 10. Criar dataframe final com os clusters
df_final = df[['Município'] + indicadores_para_analise + ['cluster']].copy()

# Visualização: boxplot por cluster para cada indicador
plt.figure(figsize=(16, 12))
for i, col in enumerate(indicadores_para_analise):
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='cluster', y=col, data=df_final)
    plt.title(col)
    plt.ylabel('Valor')
    plt.xlabel('Cluster')
plt.tight_layout()
plt.savefig('indicadores_por_cluster.png')
plt.close()

# Salvar os resultados
df_final.to_csv('municipios_clusters_sem_idhm.csv', index=False)

# Salvar estatísticas por cluster
with open('estatisticas_clusters_sem_idhm.txt', 'w') as f:
    f.write("Estatísticas dos clusters (valores originais):\n")
    f.write(cluster_stats.to_string())
    
    f.write("\n\nQuantidade de municípios em cada cluster:\n")
    for cluster, count in contagem_por_cluster.items():
        f.write(f"Cluster {cluster}: {count} municípios\n")

# Criar uma visualização radar chart para comparar os clusters
# Primeiro, obtemos os valores médios de cada indicador por cluster
cluster_means = df.groupby('cluster')[indicadores_para_analise].mean()

# Normalizar os valores para o radar chart
radar_df = cluster_means.copy()
for col in radar_df.columns:
    radar_df[col] = (radar_df[col] - radar_df[col].min()) / (radar_df[col].max() - radar_df[col].min())

# Plotar radar chart
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

def radar_chart(df, title):
    # Número de variáveis
    categories = list(df.columns)
    N = len(categories)
    
    # Ângulos para cada eixo
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Fechar o círculo
    
    # Inicializar a figura
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    # Adicionar linhas de grade
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    
    # Definir os rótulos
    plt.xticks(angles[:-1], categories, size=12)
    
    # Desenhar os limites do gráfico
    ax.set_rlabel_position(0)
    plt.yticks([0.2, 0.4, 0.6, 0.8], ["0.2", "0.4", "0.6", "0.8"], color="grey", size=10)
    plt.ylim(0, 1)
    
    # Plotar os dados para cada cluster
    for i, row in df.iterrows():
        values = row.values.flatten().tolist()
        values += values[:1]  # Fechar o círculo
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=f"Cluster {i}")
        ax.fill(angles, values, alpha=0.1)
    
    # Adicionar legenda
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title(title, size=15, y=1.1)
    
    return fig

radar_fig = radar_chart(radar_df, 'Comparação de Clusters - Indicadores Normalizados (sem IDHM)')
radar_fig.savefig('radar_clusters_sem_idhm.png', bbox_inches='tight')
plt.close()

print(f"Análise concluída. Os municípios foram classificados em {n_clusters} clusters sem usar IDHM 2010.")
print(f"Arquivos salvos:\n- municipios_clusters_sem_idhm.csv (todos os dados)\n- estatisticas_clusters_sem_idhm.txt (resumo estatístico)")
print(f"- Um arquivo CSV para cada cluster (cluster_X_detalhes.csv)")
print(f"- Visualizações: distribuicao_original.png, outliers_pos_log.png, n_clusters.png, clusters_pca.png, indicadores_por_cluster.png, radar_clusters_sem_idhm.png")

Estatísticas dos dados antes da normalização:
       IDEB AI - Pública  IDEB Anos Finais - Pública  \
count         224.000000                  224.000000   
mean            5.415766                    4.660455   
std             0.880009                    0.661246   
min             3.500000                    2.600000   
25%             4.800000                    4.200000   
50%             5.300000                    4.700000   
75%             5.900000                    5.100000   
max             8.400000                    6.500000   

       Taxa de evasão ensino fund. Anos Iniciais  \
count                                 224.000000   
mean                                    1.437500   
std                                     1.261903   
min                                     0.000000   
25%                                     0.675000   
50%                                     1.200000   
75%                                     1.825000   
max                              

# Mapa 

In [57]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import numpy as np
from datetime import datetime

# -------------------------------
# 1. Carregar os dados gerados pelo KMeans
# -------------------------------
# Atualizado para usar o arquivo sem IDHM
df_clusters = pd.read_csv('municipios_clusters.csv')

# Listar as colunas disponíveis
print("Colunas disponíveis:", df_clusters.columns.tolist())

# Verificar os valores únicos de cluster
print("\nValores únicos de cluster:", df_clusters['cluster'].unique())
print(f"Total de municípios por cluster:")
print(df_clusters['cluster'].value_counts().sort_index())

# Normalizar os nomes dos municípios para facilitar a junção com o shapefile
df_clusters['Município'] = df_clusters['Município'].str.upper().str.normalize('NFKD')\
    .str.encode('ascii', errors='ignore').str.decode('utf-8')

# Usar o nome correto da coluna de cluster
cluster_column = 'cluster'

# Converter cluster para string para garantir compatibilidade na visualização
df_clusters[cluster_column] = df_clusters[cluster_column].astype(str)

# Criar um mapeamento para exibição dos clusters começando do 1 em vez do 0
cluster_display_map = {'0': '1', '1': '2', '2': '3', '3': '4', '4': '5'}
df_clusters['cluster_display'] = df_clusters[cluster_column].map(cluster_display_map)

# Calcular estatísticas por cluster para análise
# Removido o IDHM 2010 das estatísticas
cluster_stats = df_clusters.groupby(cluster_column).agg({
    'IDEB AI - Pública': 'mean',
    'IDEB Anos Finais - Pública': 'mean',
    'Taxa de evasão ensino fund. Anos Iniciais': 'mean',
    'Taxa de evasão ensino fund. Anos Finais': 'mean',
    'Taxa de Abandono - Anos Iniciais': 'mean',
    'Taxa de Abandono - Anos Finais': 'mean'
})

print("\nEstatísticas por cluster:")
print(cluster_stats)

# Identificar as características distintivas de cada cluster baseado na nova análise sem IDHM
# Estas descrições são baseadas nas estatísticas fornecidas
cluster_descriptions = {
    '0': 'IDEB Bom e Evasão Moderada',
    '1': 'Alta Taxa de Abandono',
    '2': 'IDEB Excelente e Baixo Abandono',
    '3': 'Baixa Evasão e Abandono Moderado',
    '4': 'IDEB Médio e Evasão Alta'
}

# Verificar quais clusters realmente existem nos dados
existing_clusters = df_clusters[cluster_column].unique()
print("Clusters existentes nos dados:", existing_clusters)

# Garantir que todos os clusters têm uma descrição
for cluster in existing_clusters:
    if cluster not in cluster_descriptions:
        # Usar o mapeamento para exibir o número do cluster começando em 1
        display_num = cluster_display_map.get(cluster, str(int(cluster) + 1))
        cluster_descriptions[cluster] = f"Cluster {display_num}"

df_clusters['Cluster_Descricao'] = df_clusters[cluster_column].map(cluster_descriptions)

# Função para calcular o número de municípios em cada cluster
def contar_municipios(dataframe, coluna_cluster):
    """Retorna um dicionário com a contagem de municípios por cluster"""
    contagem = dataframe[coluna_cluster].value_counts().to_dict()
    # Garantir que as chaves são strings
    return {str(k): v for k, v in contagem.items()}

# Calcular dinamicamente as contagens de municípios por cluster
cluster_counts = contar_municipios(df_clusters, cluster_column)
print("\nContagem de municípios por cluster:")
for cluster_id, count in cluster_counts.items():
    display_id = cluster_display_map.get(cluster_id, str(int(cluster_id) + 1))
    print(f"Cluster {display_id}: {count} municípios")

# -------------------------------
# 2. Carregar o shapefile dos municípios
# -------------------------------
shapefile_path = 'media/PI_Municipios_2022.shp'  # Ajuste o caminho conforme necessário
try:
    gdf = gpd.read_file(shapefile_path)
    print(f"Shapefile carregado com sucesso. Total de {len(gdf)} municípios.")
except Exception as e:
    print(f"Erro ao carregar shapefile: {e}")
    print("Verifique se o arquivo existe no caminho especificado.")
    exit(1)

# Normalizar os nomes dos municípios no shapefile
gdf['NM_MUN'] = gdf['NM_MUN'].str.upper().str.normalize('NFKD')\
    .str.encode('ascii', errors='ignore').str.decode('utf-8')

# -------------------------------
# 3. Mesclar os clusters com o GeoDataFrame
# -------------------------------
# Selecionar as colunas relevantes para o mapa
colunas_para_mapa = ['Município', cluster_column, 'Cluster_Descricao', 'cluster_display']

# Adicionar as colunas de indicadores para hover data - Removido IDHM 2010
indicadores_educacionais = [
    'IDEB AI - Pública',
    'IDEB Anos Finais - Pública',
    'Taxa de evasão ensino fund. Anos Iniciais',
    'Taxa de evasão ensino fund. Anos Finais',
    'Taxa de Abandono - Anos Iniciais',
    'Taxa de Abandono - Anos Finais'
]

for col in indicadores_educacionais:
    if col in df_clusters.columns:
        colunas_para_mapa.append(col)

# Realizar a fusão dos dados
gdf_merged = gdf.merge(df_clusters[colunas_para_mapa],
                      left_on='NM_MUN', right_on='Município', how='left')

# Verificar quantos municípios foram mesclados com sucesso
matched_count = gdf_merged[~gdf_merged[cluster_column].isna()].shape[0]
total_count = gdf_merged.shape[0]
print(f"Municípios mesclados com sucesso: {matched_count} de {total_count} ({matched_count/total_count*100:.1f}%)")

# Listar municípios que não foram encontrados na junção
if matched_count < total_count:
    unmatched = gdf_merged[gdf_merged[cluster_column].isna()]['NM_MUN'].tolist()
    print(f"Municípios não encontrados nos dados de cluster ({len(unmatched)}):")
    print(", ".join(unmatched[:10]) + ("..." if len(unmatched) > 10 else ""))

# -------------------------------
# 4. Criar um dicionário de cores para os clusters
# -------------------------------
# Cores personalizadas por cluster - cores mais distintas para maior clareza
custom_colors = {
    '0': '#1f77b4',  # Azul - Cluster 1
    '1': '#ff7f0e',  # Laranja - Cluster 2
    '2': '#2ca02c',  # Verde - Cluster 3
    '3': '#d62728',  # Vermelho - Cluster 4
    '4': '#9467bd',  # Roxo - Cluster 5
}

# Garantir que temos cores para todos os clusters
for cluster in existing_clusters:
    if cluster not in custom_colors:
        # Gerar uma cor aleatória para clusters não mapeados
        custom_colors[cluster] = f'#{np.random.randint(0, 16777215):06x}'

# -------------------------------
# 5. Preparar dados para hover
# -------------------------------
# Criar colunas formatadas para exibição no hover
hover_columns = {}

# Definir columns para hover com formatação adequada
for col in indicadores_educacionais:
    if col in gdf_merged.columns:
        # Criar uma versão formatada do indicador para o hover
        format_col = col.replace(' ', '_')
        if 'Taxa' in col or 'taxa' in col:
            gdf_merged[format_col] = gdf_merged[col].fillna(0).apply(lambda x: f"{x:.2f}%")
        else:
            gdf_merged[format_col] = gdf_merged[col].fillna(0).apply(lambda x: f"{x:.2f}")
        hover_columns[format_col] = True
        # Remover a coluna original do hover
        hover_columns[col] = False

# Adicionar a descrição do cluster ao hover
hover_columns['Cluster_Descricao'] = True
# Usar o número do cluster para exibição (começando de 1)
hover_columns['cluster_display'] = True
# Ocultar o número original do cluster
hover_columns[cluster_column] = False

# -------------------------------
# 6. Criar o mapa interativo
# -------------------------------
try:
    fig = px.choropleth_mapbox(
        gdf_merged,
        geojson=gdf_merged.geometry,
        locations=gdf_merged.index,
        color=cluster_column,  # Continuamos usando o cluster original para manter a consistência das cores
        color_discrete_map=custom_colors,
        hover_name='NM_MUN',
        hover_data=hover_columns,
        mapbox_style="carto-positron",
        center={"lat": -7.718, "lon": -42.728},
        zoom=5,
        opacity=0.7,
        labels={
            'Cluster_Descricao': 'Perfil',
            'IDEB_AI_-_Pública': 'IDEB Anos Iniciais',
            'IDEB_Anos_Finais_-_Pública': 'IDEB Anos Finais',
            'Taxa_de_evasão_ensino_fund._Anos_Iniciais': 'Evasão AI (%)',
            'Taxa_de_evasão_ensino_fund._Anos_Finais': 'Evasão AF (%)',
            'Taxa_de_Abandono_-_Anos_Iniciais': 'Abandono AI (%)',
            'Taxa_de_Abandono_-_Anos_Finais': 'Abandono AF (%)',
            'cluster_display': 'Cluster'
        }
    )

    # -------------------------------
    # 7. Ajustar a legenda e exibição
    # -------------------------------
    # Usar as contagens calculadas dinamicamente
    cluster_counts = contar_municipios(df_clusters, cluster_column)
    
    # Criar títulos para legendas com contagens e números de cluster ajustados
    cluster_legend_labels = {}
    for cluster, description in cluster_descriptions.items():
        # Obter a contagem do dicionário calculado dinamicamente
        count = cluster_counts.get(cluster, 0)
            
        # Usar o número mapeado do cluster (começando de 1)
        display_num = cluster_display_map.get(cluster, str(int(cluster) + 1))
        cluster_legend_labels[cluster] = f"Cluster {display_num}: {description} ({count} municípios)"
    
    # Atualizar layout com legendas melhoradas
    fig.update_layout(
        legend_title_text='Perfil Educacional',
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.95,  # Move legend closer to top
            xanchor="left",
            x=0.01,
            itemsizing="constant",
            font=dict(size=8),  # Reduce font size
            bgcolor='rgba(255,255,255,0.7)',  # Semi-transparent background
            bordercolor='rgba(0,0,0,0.2)',  # Light border
            borderwidth=1
        ),
        margin={"r": 0, "t": 30, "l": 0, "b": 0},
        title={
            'text': "Classificação de Municípios por Indicadores Educacionais - 5 Clusters (sem IDHM)",
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 12}  # Slightly reduce title font size
        }
    )
    
    # Atualizar os nomes das legendas para incluir contagens e usar numeração a partir de 1
    for i, trace in enumerate(fig.data):
        cluster_id = trace.name
        if cluster_id in cluster_legend_labels:
            fig.data[i].name = cluster_legend_labels[cluster_id]

    # -------------------------------
    # 8. Adicionar resumo das características dos clusters de forma mais compacta
    # -------------------------------
    # Criar versões mais curtas e concisas das análises dos clusters
    def get_cluster_analysis_compact(cluster_id):
        if cluster_id == '0':
            return "IDEB: 5.60 AI, 4.94 AF. Abandono: 0.11% AI, 0.59% AF. Evasão: 1.64% AI, 4.06% AF."
        elif cluster_id == '1':
            return "IDEB: 4.64 AI, 3.93 AF. Abandono: 1.68% AI, 4.25% AF. Evasão: 2.94% AI, 5.16% AF."
        elif cluster_id == '2':
            return "IDEB: 6.51 AI, 5.30 AF. Abandono: 0.03% AI, 0.11% AF. Evasão: 0.59% AI, 2.54% AF."
        elif cluster_id == '3':
            return "IDEB: 4.84 AI, 4.20 AF. Abandono: 0.31% AI, 1.33% AF. Evasão: 0.48% AI, 1.12% AF."
        elif cluster_id == '4':
            return "IDEB: 4.75 AI, 4.20 AF. Abandono: 0.23% AI, 1.91% AF. Evasão: 1.47% AI, 4.18% AF."
        else:
            return "Análise não disponível."
    
    # Gerar o texto do resumo de forma mais compacta
    summary_text = "<b>Resumo dos Clusters (sem IDHM):</b><br>"
    for cluster_id in sorted(existing_clusters):
        display_num = cluster_display_map.get(cluster_id, str(int(cluster_id) + 1))
        count = cluster_counts.get(cluster_id, 0)
        analysis = get_cluster_analysis_compact(cluster_id)
        summary_text += f"<b>Cluster {display_num}:</b> {cluster_descriptions[cluster_id]} ({count} mun.) - {analysis}<br>"
    
    # Adicionar como anotação de texto no mapa com tamanho reduzido
    fig.add_annotation(
        xref="paper", yref="paper",
        x=0.5, y=-0.03,  # Move it closer to the map
        text=summary_text,
        showarrow=False,
        font=dict(size=7),  # Smaller font
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="rgba(0,0,0,0.2)",
        borderwidth=1,
        align="left",
        width=700,  # Constrain width
        xanchor="center",
        yanchor="top"
    )
    
    # Salvar o mapa interativo com nome incluindo data
    date_str = datetime.now().strftime("%Y%m%d")
    output_file = f"mapa_interativo_5clusters_educacao_sem_idhm_{date_str}.html"
    fig.write_html(output_file)
    print(f"Mapa interativo salvo como '{output_file}'")
    
    # No Jupyter ou ambiente que suporte exibição, você pode chamar fig.show()
    # fig.show()
    
except Exception as e:
    print(f"Erro ao criar o mapa: {e}")
    import traceback
    traceback.print_exc()
    print("Verifique se os dados estão formatados corretamente e se o GeoPandas está instalado.")

Colunas disponíveis: ['Município', 'IDEB AI - Pública', 'IDEB Anos Finais - Pública', 'Taxa de evasão ensino fund. Anos Iniciais', 'Taxa de evasão ensino fund. Anos Finais', 'Taxa de Abandono - Anos Iniciais', 'Taxa de Abandono - Anos Finais', 'cluster']

Valores únicos de cluster: [2 0 1 4 3]
Total de municípios por cluster:
cluster
0    78
1    28
2    48
3    20
4    50
Name: count, dtype: int64

Estatísticas por cluster:
         IDEB AI - Pública  IDEB Anos Finais - Pública  \
cluster                                                  
0                 5.595074                    4.940519   
1                 4.639849                    3.930731   
2                 6.506250                    5.303343   
3                 4.840000                    4.195000   
4                 4.754000                    4.201209   

         Taxa de evasão ensino fund. Anos Iniciais  \
cluster                                              
0                                         1.644872   
1 

# Diagnostico 

In [55]:
import pandas as pd
import numpy as np

# Carregar os dados do arquivo municipios_clusters_sem_idhm.csv
try:
    # Carregar os dados gerados pelo K-means sem IDHM
    df_clusters = pd.read_csv('municipios_clusters.csv')
    print(f"Arquivo carregado com sucesso. Total de {len(df_clusters)} municípios.")
except Exception as e:
    print(f"Erro ao carregar o arquivo: {e}")
    print("Verifique se o arquivo existe no diretório atual.")
    exit(1)

# Identificar a coluna de cluster
cluster_column = 'cluster'
if cluster_column not in df_clusters.columns:
    # Tentar encontrar coluna alternativa
    for col in df_clusters.columns:
        if 'cluster' in col.lower():
            cluster_column = col
            print(f"Usando '{cluster_column}' como coluna de cluster.")
            break
    else:
        print("Erro: Não foi possível encontrar a coluna de cluster.")
        exit(1)

# Converter cluster para int se possível
try:
    df_clusters[cluster_column] = df_clusters[cluster_column].astype(int)
except:
    print("Aviso: Não foi possível converter os clusters para números inteiros.")

# Identificar as colunas de indicadores educacionais (removido IDHM 2010)
colunas_indicadores = [
    'IDEB AI - Pública',
    'IDEB Anos Finais - Pública',
    'Taxa de evasão ensino fund. Anos Iniciais',
    'Taxa de evasão ensino fund. Anos Finais',
    'Taxa de Abandono - Anos Iniciais',
    'Taxa de Abandono - Anos Finais'
]

# Verificar quais colunas existem no DataFrame
colunas_existentes = []
for col in colunas_indicadores:
    # Verificar também colunas com espaço no final
    col_trim = col.rstrip()
    matching_cols = [c for c in df_clusters.columns if c.rstrip() == col_trim]
    
    if col in df_clusters.columns:
        colunas_existentes.append(col)
    elif matching_cols:
        colunas_existentes.append(matching_cols[0])

print(f"Colunas de indicadores encontradas: {colunas_existentes}")

# Calcular contagem de municípios por cluster
cluster_counts = df_clusters[cluster_column].value_counts().sort_index()
print("\nQuantidade de municípios por cluster:")
print(cluster_counts)

# Calcular valores médios para cada cluster
medias_por_cluster = df_clusters.groupby(cluster_column)[colunas_existentes].mean()
print("\nValores médios por cluster:")
print(medias_por_cluster)

# Formatando nomes mais amigáveis para indicadores
nomes_formatados = {
    'IDEB AI - Pública': 'IDEB Anos Iniciais',
    'IDEB Anos Finais - Pública': 'IDEB Anos Finais',
    'Taxa de evasão ensino fund. Anos Iniciais': 'Taxa de Evasão AI',
    'Taxa de evasão ensino fund. Anos Finais': 'Taxa de Evasão AF',
    'Taxa de Abandono - Anos Iniciais': 'Taxa de Abandono AI',
    'Taxa de Abandono - Anos Finais': 'Taxa de Abandono AF'
}

# Ajustar para colunas com espaço no final
for col in colunas_existentes:
    col_trim = col.rstrip()
    if col_trim in nomes_formatados and col != col_trim:
        nomes_formatados[col] = nomes_formatados[col_trim]

# Ajustar nomes das colunas para exibição
medias_display = medias_por_cluster.rename(columns=nomes_formatados)

# Preparar DataFrame para exportação
data_export = {
    "cluster": medias_por_cluster.index.tolist()
}

# Adicionar médias dos indicadores ao dicionário de exportação
for col in colunas_existentes:
    data_export[col] = medias_por_cluster[col].tolist()

# Descrições melhoradas para cada cluster com base nas estatísticas atualizadas
cluster_nomes = {
    0: "IDEB Bom e Evasão Moderada",
    1: "Alta Taxa de Abandono",
    2: "IDEB Excelente e Baixo Abandono",
    3: "Baixa Evasão e Abandono Moderado",
    4: "IDEB Médio e Evasão Alta"
}

# Função para gerar descrição do perfil de cada cluster
def get_perfil_cluster(cluster):
    """Retorna uma descrição detalhada do perfil do cluster baseada nas estatísticas atualizadas."""
    if cluster == 0:
        return "Municípios com bom desempenho nos IDEBs, baixas taxas de abandono e evasão moderada, representando um perfil educacional equilibrado."
    elif cluster == 1:
        return "Municípios com os maiores desafios educacionais, apresentando altas taxas de abandono, evasão elevada e IDEB abaixo da média."
    elif cluster == 2:
        return "Municípios de excelência educacional, com os melhores IDEBs e as menores taxas de abandono e evasão, demonstrando qualidade e retenção escolar."
    elif cluster == 3:
        return "Municípios com taxas de evasão excepcionalmente baixas, IDEB moderado e abandono controlado, indicando boa capacidade de retenção escolar."
    elif cluster == 4:
        return "Municípios com IDEB na média, baixo abandono nos Anos Iniciais, mas desafios significativos na evasão e abandono nos Anos Finais."
    else:
        return f"Cluster {cluster} - Perfil não definido"

# Valores de referência nacionais para comparação
referencias = {
    'IDEB AI - Pública': 5.8,  # Média nacional IDEB Anos Iniciais 2023
    'IDEB Anos Finais - Pública': 4.8,  # Média nacional IDEB Anos Finais 2023
    'Taxa média de evasão': 3.0,  # Valor de referência aproximado
    'Taxa média de abandono': 3.0  # Valor de referência aproximado
}

# Criar diagnóstico para cada cluster
clusters_unicos = medias_por_cluster.index.tolist()
diagnostico = {}

for cluster in clusters_unicos:
    # Lista para armazenar pontos fortes, fracos e recomendações
    pontos_fortes = []
    pontos_fracos = []
    recomendacoes = []
    
    # Extrair dados relevantes para todos os clusters
    if cluster in medias_por_cluster.index:
        ideb_ai = medias_por_cluster.loc[cluster, 'IDEB AI - Pública']
        ideb_af = medias_por_cluster.loc[cluster, 'IDEB Anos Finais - Pública']
        evasao_ai = medias_por_cluster.loc[cluster, 'Taxa de evasão ensino fund. Anos Iniciais']
        evasao_af = medias_por_cluster.loc[cluster, 'Taxa de evasão ensino fund. Anos Finais']
        abandono_ai = medias_por_cluster.loc[cluster, 'Taxa de Abandono - Anos Iniciais']
        abandono_af = medias_por_cluster.loc[cluster, 'Taxa de Abandono - Anos Finais']
    else:
        print(f"Aviso: Cluster {cluster} não encontrado nos dados.")
        continue
    
    # Análise específica para cada cluster baseada nas estatísticas corretas
    if cluster == 0:  # IDEB Bom e Evasão Moderada
        # Pontos fortes
        pontos_fortes = [
            f"- IDEB Anos Iniciais satisfatório ({ideb_ai:.2f})",
            f"- IDEB Anos Finais acima da média do estado ({ideb_af:.2f})",
            f"- Taxas de abandono muito baixas (AI: {abandono_ai:.2f}%, AF: {abandono_af:.2f}%)",
            "- Equilíbrio entre desempenho e permanência escolar"
        ]
        
        # Pontos fracos
        pontos_fracos = [
            f"- Taxa de evasão nos Anos Finais significativa ({evasao_af:.2f}%)",
            "- Potencial para melhorar ainda mais o IDEB Anos Iniciais",
            "- Discrepância entre taxas de evasão dos Anos Iniciais e Finais",
            "- Necessidade de fortalecer a transição entre ciclos"
        ]
        
        # Recomendações específicas
        recomendacoes = [
            "- Implementar acompanhamento personalizado para estudantes na transição para Anos Finais",
            "- Desenvolver programa de monitoramento e redução de evasão",
            "- Documentar e disseminar boas práticas que contribuem para baixas taxas de abandono",
            "- Criar sistema de alerta precoce para potenciais casos de evasão"
        ]
        
    elif cluster == 1:  # Alta Taxa de Abandono
        # Pontos fortes
        pontos_fortes = [
            "- Grupo claramente identificado para intervenções de alto impacto",
            "- Potencial para melhoria significativa com programas focados e recursos direcionados",
            f"- Alguma capacidade educacional demonstrada (IDEB AI: {ideb_ai:.2f})",
            "- Oportunidade para parcerias intersetoriais com grande potencial de transformação"
        ]
        
        # Pontos fracos
        pontos_fracos = [
            f"- Taxas de abandono extremamente elevadas (AI: {abandono_ai:.2f}%, AF: {abandono_af:.2f}%)",
            f"- Altas taxas de evasão em ambos os ciclos (AI: {evasao_ai:.2f}%, AF: {evasao_af:.2f}%)",
            f"- IDEB Anos Finais muito baixo ({ideb_af:.2f})",
            "- Desafios sistêmicos que afetam todo o ciclo educacional"
        ]
        
        # Recomendações específicas
        recomendacoes = [
            "- Implementar programa de busca ativa de estudantes com abandono escolar",
            "- Criar rede de apoio integrada (educação, assistência social, saúde) para famílias vulneráveis",
            "- Desenvolver estratégias específicas para melhorar o IDEB Anos Finais",
            "- Estabelecer metas progressivas de redução das taxas de abandono e evasão",
            "- Implantar programa de acompanhamento individual para estudantes em risco"
        ]
        
    elif cluster == 2:  # IDEB Excelente e Baixo Abandono
        # Pontos fortes
        pontos_fortes = [
            f"- IDEB Anos Iniciais excepcional ({ideb_ai:.2f}, bem acima da média nacional)",
            f"- IDEB Anos Finais destacado ({ideb_af:.2f}, acima da média nacional)",
            f"- Taxas de abandono praticamente inexistentes (AI: {abandono_ai:.2f}%, AF: {abandono_af:.2f}%)",
            f"- Taxas de evasão muito baixas, especialmente nos Anos Iniciais ({evasao_ai:.2f}%)"
        ]
        
        # Pontos fracos
        pontos_fracos = [
            f"- Taxa de evasão nos Anos Finais ainda presente ({evasao_af:.2f}%)",
            "- Desafio de manter e ampliar os excelentes resultados já alcançados",
            "- Potencial risco de acomodação devido aos bons resultados",
            "- Necessidade de estratégias específicas para a sustentabilidade dos resultados"
        ]
        
        # Recomendações específicas
        recomendacoes = [
            "- Documentar e formalizar as práticas de excelência para criação de modelo replicável",
            "- Criar programa de mentoria para outros municípios do estado",
            "- Desenvolver estratégias para reduzir ainda mais a evasão nos Anos Finais",
            "- Estabelecer metas de inovação pedagógica para contínuo aprimoramento",
            "- Implementar plano de sustentabilidade de longo prazo para os bons resultados"
        ]
        
    elif cluster == 3:  # Baixa Evasão e Abandono Moderado
        # Pontos fortes
        pontos_fortes = [
            f"- Taxa de evasão excepcionalmente baixa em ambos os ciclos (AI: {evasao_ai:.2f}%, AF: {evasao_af:.2f}%)",
            "- Sistema eficaz de retenção escolar e acompanhamento de frequência",
            f"- IDEB compatível com a realidade local (AI: {ideb_ai:.2f}, AF: {ideb_af:.2f})",
            "- Boa continuidade do fluxo escolar entre Anos Iniciais e Finais"
        ]
        
        # Pontos fracos
        pontos_fracos = [
            f"- IDEB abaixo da média nacional em ambos os ciclos",
            f"- Taxa de abandono moderada nos Anos Finais ({abandono_af:.2f}%)",
            "- Potencial foco excessivo na frequência versus qualidade do ensino",
            "- Oportunidade de melhoria nos indicadores de aprendizagem"
        ]
        
        # Recomendações específicas
        recomendacoes = [
            "- Documentar e compartilhar as estratégias eficazes de retenção de alunos",
            "- Desenvolver ações específicas para elevar o IDEB, mantendo as baixas taxas de evasão",
            "- Implementar monitoramento da qualidade das aulas e formação continuada para professores",
            "- Criar programa de redução do abandono nos Anos Finais",
            "- Estabelecer metas progressivas de melhoria do IDEB"
        ]
        
    elif cluster == 4:  # IDEB Médio e Evasão Alta
        # Pontos fortes
        pontos_fortes = [
            f"- IDEB razoável considerando os desafios (AI: {ideb_ai:.2f}, AF: {ideb_af:.2f})",
            f"- Taxa de abandono relativamente baixa nos Anos Iniciais ({abandono_ai:.2f}%)",
            "- Capacidade de manter estudantes nos primeiros anos do ensino fundamental",
            "- Potencial para melhorias significativas com intervenções focadas"
        ]
        
        # Pontos fracos
        pontos_fracos = [
            f"- Taxa de evasão elevada nos Anos Finais ({evasao_af:.2f}%)",
            f"- Taxa de abandono significativa nos Anos Finais ({abandono_af:.2f}%)",
            "- Fragilidade na transição entre Anos Iniciais e Finais",
            "- Desafios na manutenção da qualidade e permanência nos Anos Finais"
        ]
        
        # Recomendações específicas
        recomendacoes = [
            "- Desenvolver programa especial de transição entre os ciclos do ensino fundamental",
            "- Implementar sistema de alerta precoce para identificar estudantes em risco de evasão",
            "- Criar estratégias específicas para redução do abandono nos Anos Finais",
            "- Estabelecer programa de apoio acadêmico e socioemocional para adolescentes",
            "- Promover formação continuada para professores dos Anos Finais"
        ]
    
    # Adicionar ao dicionário de diagnósticos
    diagnostico[cluster] = {
        "Perfil": get_perfil_cluster(cluster),
        "Nome do Cluster": cluster_nomes.get(cluster, f"Cluster {cluster}"),
        "Pontos Fortes": "\n".join(pontos_fortes) if pontos_fortes else "- Nenhum indicador destaca-se positivamente",
        "Pontos Fracos": "\n".join(pontos_fracos) if pontos_fracos else "- Nenhum indicador destaca-se negativamente",
        "Recomendações": "\n".join(recomendacoes)
    }

# Adicionar diagnóstico ao DataFrame
df_diagnostico = pd.DataFrame()
df_diagnostico["cluster"] = data_export["cluster"]
df_diagnostico["Nome do Cluster"] = df_diagnostico["cluster"].map(lambda x: cluster_nomes.get(x, f"Cluster {x}"))
df_diagnostico["Perfil"] = df_diagnostico["cluster"].map(lambda x: diagnostico.get(x, {}).get("Perfil", ""))
df_diagnostico["Pontos Fortes"] = df_diagnostico["cluster"].map(lambda x: diagnostico.get(x, {}).get("Pontos Fortes", ""))
df_diagnostico["Pontos Fracos"] = df_diagnostico["cluster"].map(lambda x: diagnostico.get(x, {}).get("Pontos Fracos", ""))
df_diagnostico["Recomendações"] = df_diagnostico["cluster"].map(lambda x: diagnostico.get(x, {}).get("Recomendações", ""))

# Adicionar dados dos indicadores
for col in colunas_existentes:
    df_diagnostico[col] = data_export[col]

# Exportar para CSV
try:
    df_diagnostico.to_csv("diagnostico_clusters_educacao_sem_idhm.csv", index=False)
    print("\nDiagnóstico dos clusters educacionais salvo como 'diagnostico_clusters_educacao_sem_idhm.csv'")
except Exception as e:
    print(f"Erro ao salvar o diagnóstico: {e}")

# Também criar um CSV com médias mais limpo para exibição
df_medias = pd.DataFrame({
    "cluster": clusters_unicos,
    "quantidade_municipios": [cluster_counts.get(c, 0) for c in clusters_unicos],
    "nome_cluster": [cluster_nomes.get(c, f"Cluster {c}") for c in clusters_unicos],
    "perfil": [get_perfil_cluster(c) for c in clusters_unicos]
})

# Adicionar as médias formatadas
for col in colunas_existentes:
    nome_curto = nomes_formatados.get(col, col)
    
    if 'IDEB' in col:
        df_medias[nome_curto] = [f"{medias_por_cluster.loc[c, col]:.2f}" for c in clusters_unicos]
    else:  # Taxas
        df_medias[nome_curto] = [f"{medias_por_cluster.loc[c, col]:.2f}%" for c in clusters_unicos]

# Exportar para CSV
try:
    df_medias.to_csv("medias_clusters_educacao_sem_idhm.csv", index=False)
    print("Médias formatadas salvas como 'medias_clusters_educacao_sem_idhm.csv'")
except Exception as e:
    print(f"Erro ao salvar as médias: {e}")

# Mostrar um resumo do diagnóstico
print("\nResumo do diagnóstico por cluster (análise sem IDHM):")
for cluster in clusters_unicos:
    if cluster in diagnostico:
        cluster_info = diagnostico[cluster]
        print(f"\nCluster {cluster + 1} - {cluster_info['Nome do Cluster']} ({cluster_counts.get(cluster, 0)} municípios):")
        print(f"Perfil: {cluster_info['Perfil']}")
        print(f"Pontos Fortes:\n{cluster_info['Pontos Fortes']}")
        print(f"Pontos Fracos:\n{cluster_info['Pontos Fracos']}")
        print(f"Recomendações:\n{cluster_info['Recomendações']}")

Arquivo carregado com sucesso. Total de 224 municípios.
Colunas de indicadores encontradas: ['IDEB AI - Pública', 'IDEB Anos Finais - Pública', 'Taxa de evasão ensino fund. Anos Iniciais', 'Taxa de evasão ensino fund. Anos Finais', 'Taxa de Abandono - Anos Iniciais', 'Taxa de Abandono - Anos Finais']

Quantidade de municípios por cluster:
cluster
0    78
1    28
2    48
3    20
4    50
Name: count, dtype: int64

Valores médios por cluster:
         IDEB AI - Pública  IDEB Anos Finais - Pública  \
cluster                                                  
0                 5.595074                    4.940519   
1                 4.639849                    3.930731   
2                 6.506250                    5.303343   
3                 4.840000                    4.195000   
4                 4.754000                    4.201209   

         Taxa de evasão ensino fund. Anos Iniciais  \
cluster                                              
0                                        